In [ ]:
import os
import pandas as pd
import numpy as np
from time import time
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, concatenate, Flatten, Dense, Dropout
from keras.layers import InputLayer, Activation, Concatenate,Input
from keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import keras
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading all the files

In [ ]:
data_columns = ['user_id', 'movie_id', 'rating', 'timestamp']
item_columns = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action',
             'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir',
             'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci_Fi', 'Thriller', 'War', 'Western']
user_columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

In [ ]:
users = pd.read_csv('/content/drive/Shareddrives/Deep Learning Lab/ml-100k/u.user', sep='|', names=user_columns, encoding='latin-1')
item = pd.read_csv('/content/drive/Shareddrives/Deep Learning Lab/ml-100k/u.item', sep='|', names=item_columns, encoding='latin-1')
data = pd.read_csv('/content/drive/Shareddrives/Deep Learning Lab/ml-100k/u.data', sep='\t', names=data_columns, encoding='latin-1')

Combining the files

In [ ]:
combined_data= pd.merge(pd.merge(item, data), users)

In [ ]:
combined_data.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Childrens,...,Thriller,War,Western,user_id,rating,timestamp,age,gender,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,1,0,0,308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,308,5,887736696,60,M,retired,95076


Defining the features for the wide model

In [ ]:
wide_features = combined_data[['gender', 'occupation']]

In [ ]:
wide_features['gender_occupation'] = wide_features['gender'] + "_" + wide_features['occupation']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Encoding the features for the wide model

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(wide_features[['gender_occupation']])
one_hot_encoded_df = pd.DataFrame(encoder.transform(wide_features[['gender_occupation']]).toarray(),
                                  columns=encoder.get_feature_names())
wide_features = wide_features.join(one_hot_encoded_df)
wide_features.drop(['gender', 'occupation', 'gender_occupation'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
wide_features.head()

,x0_F_administrator,x0_F_artist,x0_F_educator,x0_F_engineer,x0_F_entertainment,x0_F_executive,x0_F_healthcare,x0_F_homemaker,x0_F_lawyer,x0_F_librarian,...,x0_M_marketing,x0_M_none,x0_M_other,x0_M_programmer,x0_M_retired,x0_M_salesman,x0_M_scientist,x0_M_student,x0_M_technician,x0_M_writer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Defining the features for the deep model

In [ ]:
deep_features = combined_data[['age', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
              'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci_Fi', 'Thriller', 'War',
              'Western', 'gender', 'occupation']]

In [ ]:
deep_features['genre'] = deep_features[['unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
                            'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci_Fi',
                            'Thriller', 'War', 'Western']].idxmax(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
deep_features.drop(columns=['unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary',
                      'Drama', 'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci_Fi', 'Thriller',
                      'War', 'Western'], axis=1, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
deep_features.head()

,age,gender,occupation,genre
0,60,M,retired,Animation
1,60,M,retired,Action
2,60,M,retired,Crime
3,60,M,retired,Drama
4,60,M,retired,Childrens


Encoding the deep model features

In [ ]:
for feature in ['gender', 'occupation', 'genre']:
    encoder = LabelEncoder()
    encoder.fit(deep_features[[feature]])
    transformed_feature = encoder.transform(deep_features[[feature]])
    deep_features[feature] = transformed_feature

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
for feature in ['age']:
    scaler = MinMaxScaler()
    scaler.fit(deep_features[[feature]])
    transformed_feature = scaler.transform(deep_features[[feature]])
    deep_features[feature] = transformed_feature

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
deep_features.head()

,age,gender,occupation,genre
0,0.80303,1,15,2
1,0.80303,1,15,0
2,0.80303,1,15,5
3,0.80303,1,15,7
4,0.80303,1,15,3


Splitting the dataset into train and test subsets

In [ ]:
X = pd.concat([wide_features, deep_features], axis=1)
y = combined_data[['rating']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def prepare_independent_variables(X):
  wide_inputs = X[['x0_F_administrator', 'x0_F_artist', 'x0_F_educator', 'x0_F_engineer',
                     'x0_F_entertainment', 'x0_F_executive', 'x0_F_healthcare',
                     'x0_F_homemaker', 'x0_F_lawyer', 'x0_F_librarian', 'x0_F_marketing',
                     'x0_F_none', 'x0_F_other', 'x0_F_programmer', 'x0_F_retired',
                     'x0_F_salesman', 'x0_F_scientist', 'x0_F_student', 'x0_F_technician',
                     'x0_F_writer', 'x0_M_administrator', 'x0_M_artist', 'x0_M_doctor',
                     'x0_M_educator', 'x0_M_engineer', 'x0_M_entertainment',
                     'x0_M_executive', 'x0_M_healthcare', 'x0_M_homemaker', 'x0_M_lawyer',
                     'x0_M_librarian', 'x0_M_marketing', 'x0_M_none', 'x0_M_other',
                     'x0_M_programmer', 'x0_M_retired', 'x0_M_salesman', 'x0_M_scientist',
                     'x0_M_student', 'x0_M_technician', 'x0_M_writer']].values
  categorical_input_1 = X[['gender']].values
  categorical_input_2 = X[['occupation']].values
  categorical_input_3 = X[['genre']].values
  numerical_input = X[['age']].values
  return wide_inputs, categorical_input_1, categorical_input_2, categorical_input_3, \
           numerical_input

In [ ]:
wide_inputs_train, categorical_input_1_train, categorical_input_2_train, \
categorical_input_3_train, numerical_input_train = prepare_independent_variables(X_train)

wide_inputs_test, categorical_input_1_test, categorical_input_2_test, \
categorical_input_3_test, numerical_input_test = prepare_independent_variables(X_test)

y_train = y_train.values
y_test = y_test.values

Creating the Model

In [ ]:
max_num_categorical_values = 50
emb_dimension = 1200
max_sequence_length = 1

In [ ]:
categorical_input_1 = Input(shape=(1,), name='categorical_input_1')
emb_1 = Embedding(input_dim=max_num_categorical_values, output_dim=emb_dimension,
                  input_length=max_sequence_length, name='emb_1')(categorical_input_1)
emb_1 = Flatten()(emb_1)
categorical_input_2 = Input(shape=(1,), name='categorical_input_2')
emb_2 = Embedding(input_dim=max_num_categorical_values, output_dim=emb_dimension,
                  input_length=max_sequence_length, name='emb_2')(categorical_input_2)
emb_2 = Flatten()(emb_2)
categorical_input_3 = Input(shape=(1,), name='categorical_input_3')
emb_3 = Embedding(input_dim=max_num_categorical_values, output_dim=emb_dimension,
                  input_length=max_sequence_length, name='emb_3')(categorical_input_3)
emb_3 = Flatten()(emb_3)
numerical_input = Input(shape=(1,), name='numerical_input')
concatenated_embeddings = concatenate([emb_1, emb_2, emb_3, numerical_input])
concatenated_embeddings = Dropout(rate=0.2)(concatenated_embeddings)

In [ ]:
x1 = Dense(1200, activation='relu')(concatenated_embeddings)
x1 = Dropout(rate=0.25)(x1)
x1 = Dense(1024, activation='relu',kernel_regularizer =tf.keras.regularizers.l2( l=0.01))(concatenated_embeddings)
x1 = Dropout(rate=0.25)(x1)
x2 = Dense(512, activation='relu',kernel_regularizer =tf.keras.regularizers.l2( l=0.01))(x1)
x2 = Dropout(rate=0.25)(x2)
x3 = Dense(256, activation='relu',kernel_regularizer =tf.keras.regularizers.l2( l=0.01))(x2)
x3 = Dropout(rate=0.25)(x3)
deep_output = Dense(64, activation='relu')(x3)

In [ ]:
num_features = len(wide_features.columns)
wide_inputs = Input(shape=(num_features,), name='wide_inputs')

In [ ]:
x = concatenate([wide_inputs, deep_output])
x = Dropout(rate=0.2)(x)

In [ ]:
wide_and_deep_output = Dense(1, activation='relu')(x)

In [ ]:
wide_and_deep_model = Model(inputs=[wide_inputs] + [categorical_input_1] + 
                            [categorical_input_2] + [categorical_input_3] + 
                            [numerical_input], outputs=wide_and_deep_output)

In [ ]:
wide_and_deep_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 categorical_input_1 (InputLaye  [(None, 1)]         0           []                               
 r)                                                                                               
                                                                                                  
 categorical_input_2 (InputLaye  [(None, 1)]         0           []                               
 r)                                                                                               
                                                                                                  
 categorical_input_3 (InputLaye  [(None, 1)]         0           []                               
 r)                                                                                         

In [ ]:
wide_and_deep_model.compile(loss='mae', optimizer=tf.keras.optimizers.Adamax(learning_rate=0.0001, decay=1e-06), metrics=['mae','accuracy'])

In [ ]:
wide_and_deep_model.fit(x={'wide_inputs': wide_inputs_train,
                           'categorical_input_1': categorical_input_1_train,
                           'categorical_input_2': categorical_input_2_train,
                           'categorical_input_3': categorical_input_3_train,
                           'numerical_input': numerical_input_train},
                        y=y_train,
                        batch_size=32, epochs=50, verbose=1,validation_split=0.2)

Epoch 1/50
2000/2000 [==============================] - 14s 7ms/step - loss: 0.9530 - mae: 0.9444 - accuracy: 0.0608 - val_loss: 0.9349 - val_mae: 0.9276 - val_accuracy: 0.0619
Epoch 2/50
2000/2000 [==============================] - 13s 7ms/step - loss: 0.9523 - mae: 0.9450 - accuracy: 0.0608 - val_loss: 0.9197 - val_mae: 0.9130 - val_accuracy: 0.0619
Epoch 3/50
2000/2000 [==============================] - 13s 7ms/step - loss: 0.9473 - mae: 0.9409 - accuracy: 0.0608 - val_loss: 0.9105 - val_mae: 0.9044 - val_accuracy: 0.0619
Epoch 4/50
2000/2000 [==============================] - 13s 7ms/step - loss: 0.9449 - mae: 0.9390 - accuracy: 0.0608 - val_loss: 0.9208 - val_mae: 0.9155 - val_accuracy: 0.0619
Epoch 5/50
2000/2000 [==============================] - 13s 7ms/step - loss: 0.9469 - mae: 0.9414 - accuracy: 0.0608 - val_loss: 0.9072 - val_mae: 0.9015 - val_accuracy: 0.0619
Epoch 6/50
2000/2000 [==============================] - 14s 7ms/step - loss: 0.9429 - mae: 0.9376 - accuracy: 0.060

In [ ]:
loss=wide_and_deep_model.evaluate(x={'wide_inputs': wide_inputs_test,
                                'categorical_input_1': categorical_input_1_test,
                                'categorical_input_2': categorical_input_2_test,
                                'categorical_input_3': categorical_input_3_test,
                                'numerical_input': numerical_input_test},
                             y=y_test,
                             batch_size=32, verbose=1)

625/625 [==============================] - 2s 3ms/step - loss: 0.9008 - mae: 0.8972 - accuracy: 0.0615


In [ ]:
predictions = wide_and_deep_model.predict(x={'wide_inputs': wide_inputs_test,
                                             'categorical_input_1': categorical_input_1_test,
                                             'categorical_input_2': categorical_input_2_test,
                                             'categorical_input_3': categorical_input_3_test,
                                             'numerical_input': numerical_input_test},
                                          batch_size=32, verbose=1)

625/625 [==============================] - 1s 2ms/step


In [ ]:
np.round(predictions)
predictions=predictions.astype(int)

In [ ]:
count=0
for i in range (0,len(predictions)):
  if predictions[i]==y_test[i]:
    count+=1

In [ ]:
print("Prediction Accuracy=",(count/len(y_test))*100)

Prediction Accuracy= 26.884999999999998
